In [ ]:
import h5py
import datetime
import numpy as np
import matplotlib.pyplot as plt
import os

# define a string which helps with selecting the data range to analyze i.e snipe_hunt_2023-07-22_10, will analyze
# data from july 22 at 10 am time-> within that hour

def compute_noise_spectral_density(time_series, sampling_rate):
    # Calculate the length of the time series
    n = len(time_series)

    # Compute the Fast Fourier Transform (FFT)
    fft_result = np.fft.fft(time_series)

    # Calculate the one-sided power spectral density
    psd = (1 / (sampling_rate * n)) * np.abs(fft_result[:n//2])**2

    # Calculate the corresponding frequencies
    freqs = np.fft.fftfreq(n, 1/sampling_rate)[:n//2]

    return freqs, np.sqrt(2*psd)

# Create the "Plots" subdirectory if it doesn't exist
if not os.path.exists("Plots"):
    os.makedirs("Plots")

# Define the directories containing the .h5 files and their corresponding calibration files
#data_directories = ["../Station_I/SnipeData", "../Station_II/SnipeData","../Station_III/SnipeData" ]
#calibration_files = ["../Station_I/coil_I_sensitivity.txt", "../Station_II/coil_II_sensitivity.txt","../Station_III/coil_III_sensitivity.txt"]

data_directories = ['./North_Jul24']
calibration_files = ['calibrationN761.csv']



#The calibration files have two fields the first if frequency in Hz, and the second is the Volt/picoTesla response
# Change to your calibration file


# Create two figures for noise spectral density and data versus timestamp
fig1, axs1 = plt.subplots(len(data_directories), figsize=(16, 9))
fig2, axs2 = plt.subplots(len(data_directories), figsize=(16, 9))

for station_index, (data_directory, calibration_file) in enumerate(zip(data_directories, calibration_files)):
    
    calibration_data = np.loadtxt(calibration_file, delimiter = ",") #v2

    frequency_calibration = calibration_data[:, 0]  # Frequency values in Hz #v2
    voltage_calibration = 10**-3 * calibration_data[:, 1] #v2
    
    # Create empty arrays to merge the data from all files for the current station
    merged_timestamps = []
    merged_data = []

    # List all files in the directory and filter for those containing "snipe_hunt_2023_07_21"
    input_files = [os.path.join(data_directory, filename) for filename in os.listdir(data_directory)]

    for input_file in input_files:
        try:
            with h5py.File(input_file, "r") as f:
                timestamps = f["timestamps"][:]
                data = f["data"][:].flatten()
                merged_timestamps.extend(timestamps)
                merged_data.extend(data)
        except OSError as e:
            print(f"Error opening file {input_file}: {e}")
            continue

    # Check if any matching files were successfully processed for the current station
    if not merged_timestamps:
        print(f"No valid files found for Station {station_index + 1}.")
    else:
        # Convert timestamps to datetime objects
        datetime_timestamps = [datetime.datetime.strptime(ts.decode(), "%Y-%m-%d %H:%M:%S.%f") for ts in merged_timestamps]

        # Calculate the number of samples in each block
        block_size = 2597

        # Define the sample rate
        #sample_rate = 2597
        # make sample rate to 2597.6
        sample_rate=2597.6

        # Calculate the number of blocks
        num_blocks = len(merged_data) // block_size

        # Create an array for the x-axis (timestamps) with the same length as the data
        unix_timestamps = [datetime_timestamps[0].timestamp()] * num_blocks
        time_range = np.linspace(unix_timestamps[0], unix_timestamps[0] + num_blocks, len(merged_data))

        # Compute the Noise Spectral Density for the merged data of the current station
        freqs, noise_spectral_density = compute_noise_spectral_density(merged_data, sample_rate)

        # Extrapolate the calibration curve to match the frequencies in FFT data
        voltage_calibration_interpolated = np.interp(np.abs(freqs), frequency_calibration, voltage_calibration)

        # Convert the amplitude to pT using the extrapolated calibration curve
        psd_data = noise_spectral_density / voltage_calibration_interpolated
        # do this until we have the calibration response
        #psd_data = noise_spectral_density

        station_label = "North" if station_index == 0 else "East"
        axs1[station_index].plot(freqs, psd_data)
        #axs1[station_index].set_xscale("log")
        axs1[station_index].set_yscale("log")
        axs1[station_index].set_xlabel("Frequency (Hz)")
        axs1[station_index].set_ylabel("Noise Spectral Density (pT/√Hz)")
        #axs1[station_index].set_ylabel("Noise Spectral Density (V/√Hz)")
        axs1[station_index].set_title(f"Noise Spectral Density Plot - Station {station_label}")
        axs1[station_index].grid(True)
        axs1[station_index].set_xlim(0.5, 5)

        # Plot the data versus timestamp in Volt for the current station on its subplot
        axs2[station_index].plot(time_range, merged_data)
        axs2[station_index].set_xlabel("Timestamp")
        axs2[station_index].set_ylabel("(V)")
        axs2[station_index].set_xlim(unix_timestamps[0] + 10., unix_timestamps[0] + 10. + 207)
        axs2[station_index].set_title(f"Data Plot - Station {station_label}")
        axs2[station_index].grid(True)

# Save each figure separately in the "Plots" subdirectory
fig1.tight_layout()
fig2.tight_layout()


fig1.savefig(os.path.join("Plots", "merged_noise_spectral_density.png"))
fig2.savefig(os.path.join("Plots", "merged_data_vs_timestamp.png"))

plt.show()


Error opening file ./North_Jul24/.DS_Store: Unable to open file (file signature not found)
